# Deep Learning Neural Network on Animal Shelter data

In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import theano

import matplotlib.pyplot as plt
import os


Using Theano backend.


In [6]:
# Set up my data directories from different machines

mac_data_dir = '/Users/christopherallison/Documents/Coding/Data'
linux_data_dir = '/home/chris/data'
win_data_dir = u'C:\\Users\\Owner\\Documents\\Data'

# Set data directory for example

data_dir = mac_data_dir

In [8]:
# Load our prepared dataset and reference data

df = pd.read_csv(os.path.join(data_dir, "prepared_dogs_df.csv"),index_col=0)

In [9]:
df.head()

,OutcomeType,SexuponOutcome,AgeuponOutcome,Color,NameLength,BreedKMeans
2,Adoption,0,2,0,6,3
5,Transfer,1,0,1,4,2
12,Adoption,0,2,2,6,3
20,Transfer,2,0,1,6,1
31,Transfer,2,0,1,6,0


In [10]:
# Drop uneccesary columns
X = df.drop('OutcomeType', axis=1)
X.dtypes

SexuponOutcome    float64
AgeuponOutcome      int64
Color               int64
NameLength          int64
BreedKMeans         int64
dtype: object

In [11]:
X.head()

,SexuponOutcome,AgeuponOutcome,Color,NameLength,BreedKMeans
2,0,2,0,6,3
5,1,0,1,4,2
12,0,2,2,6,3
20,2,0,1,6,1
31,2,0,1,6,0


In [12]:
X = X.as_matrix()

In [13]:
outcomes = df.OutcomeType.unique()

In [14]:
from sklearn import preprocessing

# This code takes our text labels and creates an encoder that we use
# To transform them into an array

encoder = preprocessing.LabelEncoder()
encoder.fit(outcomes)

encoded_y = encoder.transform(outcomes)
encoded_y

#We can also inverse_transform them back.
list(encoder.inverse_transform([0, 1, 2, 3]))

#We still need to transform the array into a matrix - this is called one hot encoding. It allows us to track the probability of each possible outcome separately.
# convert integers into dummy variables (one hot encoding)

from keras.utils import np_utils

# We'll practice with the dummy for now, but we'll transform the full labels later.
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

#We'll transform the labels into their array value, though.
df.OutcomeType = encoder.transform(df.OutcomeType)

In [15]:
train_target = np_utils.to_categorical(df['OutcomeType'].values)
train_target

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.]])

In [17]:
model = Sequential()
model.add(Dense(4, input_dim=5, init='normal', activation="relu"))
model.add(Dense(4, init='normal', activation='sigmoid'))

# Compile model
print("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

Compiling model...


In [18]:
hist = model.fit(X, train_target, validation_split=0.2)
print("")
print(hist.history)

Train on 3160 samples, validate on 790 samples
Epoch 1/10
3160/3160 [==============================] - 0s - loss: 1.2999 - acc: 0.6516 - val_loss: 1.1642 - val_acc: 0.6759
Epoch 2/10
3160/3160 [==============================] - 0s - loss: 1.0229 - acc: 0.6769 - val_loss: 0.9290 - val_acc: 0.6759
Epoch 3/10
3160/3160 [==============================] - 0s - loss: 0.9067 - acc: 0.6769 - val_loss: 0.8930 - val_acc: 0.6759
Epoch 4/10
3160/3160 [==============================] - 0s - loss: 0.8877 - acc: 0.6769 - val_loss: 0.8839 - val_acc: 0.6759
Epoch 5/10
3160/3160 [==============================] - 0s - loss: 0.8799 - acc: 0.6769 - val_loss: 0.8750 - val_acc: 0.6759
Epoch 6/10
3160/3160 [==============================] - 0s - loss: 0.8706 - acc: 0.6769 - val_loss: 0.8660 - val_acc: 0.6759
Epoch 7/10
3160/3160 [==============================] - 0s - loss: 0.8616 - acc: 0.6769 - val_loss: 0.8583 - val_acc: 0.6759
Epoch 8/10
3160/3160 [==============================] - 0s - loss: 0.8546 - ac

In [19]:
model.evaluate(X, train_target)

3744/3950 [===========================>..] - ETA: 0s

[0.84250974504253529, 0.67670886072931413]

In [20]:
model.predict_proba(X[0:2])

2/2 [==============================] - 0s


array([[ 0.21345899,  0.01125274,  0.34118161,  0.9485954 ],
       [ 0.35394636,  0.02145817,  0.21666962,  0.91691089]])